# NHS Core

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [1]:
import os
import csv

import common_core

## Runtime parameters

The "lazy" option skips files that already exist locally

In [2]:
rawPath = os.path.join(common_core.projdir, "data", "nhs-statistics", "raw")

## Configuration

Data to download from the NHS statistical work area

In [3]:
# Only using England for now due to the availability of age demographics, etc
nationNames = ["England"]

# All 7 NHS regions in England
nhsRegionNames = ["North West", "North East and Yorkshire", "Midlands",
                  "East of England", "London", "South East", "South West"]

# Combine all of these area types into a single list
areas = [("nation", ["England"]), ("nhsregion", nhsRegionNames)]

In [4]:
deathsUrl = "https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-daily-deaths/"
deathsArchiveUrl = "https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-daily-deaths/weekly-total-archive/"
vaccinationsUrl = "https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-vaccinations/"
admissionsUrl = "https://www.england.nhs.uk/statistics/statistical-work-areas/covid-19-hospital-activity/"
sitRepsUrl = "https://www.england.nhs.uk/statistics/statistical-work-areas/uec-sitrep/urgent-and-emergency-care-daily-situation-reports-2020-21/"

In [5]:
deathsFiles = [
    ("weekly", "COVID-19-total-announced-deaths-.*-weekly-file.*\.xlsx$"), # place above daily files to be picked up first
    ("weekly", "COVID-19-total-announced-deaths-.*-weekly-tables.*\.xlsx$"), # place above daily files to be picked up first
    ("daily", "COVID-19-total-announced-deaths-.*\.xlsx$"),
    ("daily", "COVID-19-daily-announced-deaths-.*\.xlsx$")
]

vaccinationsFiles = [
    ("daily", "COVID-19-daily-announced-vaccinations-.*\.xlsx$"),
    ("weekly", "COVID-19-weekly-announced-vaccinations-.*\.xlsx$"),
    ("monthly", "COVID-19-monthly-announced-vaccinations-.*\.xlsx$")
]

admissionsFiles = [
    ("daily", "COVID-19-daily-admissions-and-beds.*\.xlsx$"),
    ("weekly", "Weekly-covid-admissions-and-beds-publication.*\.xlsx$"),
    ("monthly", "Covid-Publication-.*\.xlsx$")
]

sitRepsFiles = [
    ("weekly", "UEC-Daily-SitRep-Acute-Web-File-Timeseries.*\.xlsx$")

]

sitRepYear = "2020-2021"

In [6]:
ageDemographics = [
    '00_04', '05_09', '10_14', '15_19', '20_24', '25_29',
    '30_34', '35_39', '40_44', '45_49', '50_54', '55_59',
    '60_64', '65_69', '70_74', '75_79', '80_84', '85_89', '90+'
]

## Download Data

Download spreadsheets by parsing the HTML for suitable links

In [7]:
def downloadDeaths(skipExisting=common_core.skipExisting, verbose=common_core.verbose):
    webDownload = common_core.WebDownload(skipExisting=skipExisting, verbose=verbose)
    files = webDownload.downloadFiles(rawPath, deathsUrl, deathsFiles, "deaths")
    files = webDownload.downloadFiles(rawPath, deathsArchiveUrl, deathsFiles, "deaths")
    return files

In [8]:
def downloadVaccinations(skipExisting=common_core.skipExisting, verbose=common_core.verbose):
    webDownload = common_core.WebDownload(skipExisting=skipExisting, verbose=verbose)
    files = webDownload.downloadFiles(rawPath, vaccinationsUrl, vaccinationsFiles, "vaccinations")
    return files

In [9]:
def downloadAdmissions(skipExisting=common_core.skipExisting, verbose=common_core.verbose):
    webDownload = common_core.WebDownload(skipExisting=skipExisting, verbose=verbose)
    files = webDownload.downloadFiles(rawPath, admissionsUrl, admissionsFiles, "admissions")
    return files

In [10]:
def downloadSitReps(skipExisting=False, verbose=common_core.verbose):
    webDownload = common_core.WebDownload(skipExisting=skipExisting, verbose=verbose)
    files = webDownload.downloadFiles(rawPath, sitRepsUrl, sitRepsFiles, os.path.join("sitreps", sitRepYear))
    return files